In [34]:
import pandas as pd
import sys  
sys.path.insert(0, './../')
from importlib import reload
import bbdc2021 as bbdc
reload(bbdc)

<module 'bbdc2021' from './../bbdc2021.py'>

In [23]:
mainPrediction = pd.read_csv("../submissions/submission07/challenge_submission_t08n03b2.csv")
secondPrediction = pd.read_csv("../submissions/submission03/2d_PredictionChallenge_mel.csv")

In [24]:
def addMissingSequences(mainPredDf, secondPredDf, minLength=0):
    predictionsToAdd = [] 
    secondPredDf['duration'] = secondPredDf.offset - secondPredDf.onset
    for filename in secondPredDf.filename.unique():
        mainFileRows = mainPredDf.loc[mainPredDf['filename'] == filename]
        secondFileRows = secondPredDf.loc[(secondPredDf['filename'] == filename) & (secondPredDf['duration']>=minLength)]
        for index, sequence in secondFileRows.iterrows():
            overlappingSequences = mainFileRows.loc[(mainFileRows['onset']<=sequence.offset) & (mainFileRows['offset']>=sequence.onset)]
            if len(overlappingSequences)==0:
                predictionsToAdd.append([sequence.filename, sequence.onset, sequence.offset, sequence.event_label])
    additionalPredictions = pd.DataFrame(data=predictionsToAdd, columns=['filename', 'onset', 'offset', 'event_label'])
    print("Added "+str(len(predictionsToAdd)) + " sequences")
    return mainPredDf.append(additionalPredictions)
    
predictionWithAddedSequences = bbdc.addMissingSequences(mainPrediction, secondPrediction)

Added 695 sequences


In [25]:
predictionWithAddedSequences.to_csv("../predictions/jannes/predictionAlexWithAddedSequences.csv", index=False)

In [42]:
def mergeNeighboringSequences(prediction, maxSequenceLength = 2):
    connectedPrediction = []
    mergingCounter = 0
    lastSequence = [""]
    for index, sequenceRow in prediction.iterrows():
        sequence = sequenceRow.values
        if lastSequence[0]==sequence[0] and lastSequence[3]==sequence[3] and sequence[2]-lastSequence[1]<=maxSequenceLength:
            lastSequence = [sequence[0], lastSequence[1], sequence[2], sequence[3]]
            mergingCounter +=1
            continue
        if len(lastSequence)==4:
            connectedPrediction.append(lastSequence) 
        lastSequence = sequence
    print(len(prediction.index))
    connectedDf = pd.DataFrame(data=connectedPrediction, columns=['filename', 'onset', 'offset', 'event_label'])
    print("Merged "+str(mergingCounter)+ " sequences")
    print(len(connectedDf.index))
    return connectedDf

mergeNeighboringSequences(predictionWithAddedSequences)

9442
Merged 39 sequences
39


,filename,onset,offset,event_label
0,10085_mix.wav,0.200501,2.155388,Shatter
1,10205_mix.wav,7.744361,9.548872,Cough
2,10380_mix.wav,1.027569,3.007519,Burping_and_eructation
3,10408_mix.wav,2.681704,3.909774,Burping_and_eructation
4,10499_mix.wav,6.967419,8.771930,Bark
5,10569_mix.wav,0.325815,2.180451,Doorbell
6,10634_mix.wav,7.192982,9.122807,Burping_and_eructation
7,10685_mix.wav,7.994987,9.974937,Burping_and_eructation
8,10753_mix.wav,2.932331,4.586466,Camera
9,10756_mix.wav,1.253133,3.107769,Doorbell


In [35]:
bbdc.mergeNeighboringSequences(predictionWithAddedSequences)

Merged 0 sequences


,filename,onset,offset,event_label
